Here we performed the analysis for LIME as it was better to keep it separate as it needed some small adjustment to work. The analysis has been performed for the 4 datasets and data has been saved.

In [ ]:
# We perform here the experiments for Lime importance
!pip install lime
!pip install dice-ml

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import random

import dice_ml
from dice_ml.utils import helpers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = helpers.load_adult_income_dataset()
numerical_features = dataset.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features.remove('income')
categorical_features = dataset.select_dtypes(include=['object']).columns.tolist()
target_column = 'income'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

X = dataset.drop(columns=[target_column])
y = dataset[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_cat = encoder.fit_transform(X_train[categorical_features]).todense()
X_test_cat = encoder.transform(X_test[categorical_features]).todense()


X_train_processed = np.hstack((X_train_num, np.asarray(X_train_cat)))
X_test_processed = np.hstack((X_test_num, np.asarray(X_test_cat)))

In [ ]:
X_train_processed.shape

(20838, 29)

In [ ]:
# model definition
model = RandomForestClassifier()
model.fit(X_train_processed, y_train)

RandomForestClassifier()

In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer


explainer = LimeTabularExplainer(
    training_data=X_train_processed,
    mode="classification",
    feature_names=numerical_features + encoder.get_feature_names_out(categorical_features).tolist(),
    class_names=[0, 1],
    discretize_continuous=True
)

i = 0
exp = explainer.explain_instance(
    data_row=X_test_processed[i],
    predict_fn=model.predict_proba,
    num_features=29
)


In [ ]:
feature_importance_dict = {feature: importance for feature, importance in exp.as_list()}
print(feature_importance_dict)

{'education_Doctorate <= 0.00': -0.1902496106336584, '0.00 < marital_status_Married <= 1.00': 0.1609006350291132, 'education_Prof-school <= 0.00': -0.11342070340526808, 'hours_per_week <= -0.03': -0.11005237766090797, 'education_Masters <= 0.00': -0.09211415007602437, 'education_School <= 0.00': 0.08075303459238484, 'occupation_Professional > 0.00': 0.08053455541327745, 'marital_status_Single <= 0.00': 0.06811807707866492, 'education_Bachelors > 0.00': 0.06247293188473837, 'education_HS-grad <= 0.00': 0.054415886311601734, 'occupation_White-Collar <= 0.00': -0.03588117925641065, 'marital_status_Separated <= 0.00': 0.03500379126097701, 'occupation_Blue-Collar <= 0.00': 0.034204566855246074, 'marital_status_Divorced <= 0.00': 0.034028619328218365, 'occupation_Service <= 0.00': 0.025657781126380525, 'workclass_Other/Unknown <= 0.00': 0.02323325059516983, 'workclass_Self-Employed > 0.00': 0.018204922348106143, '0.00 < gender_Female <= 1.00': -0.016726432780304556, 'occupation_Sales <= 0.00

In [ ]:
lime_importance = [-0.00665239904858031 ,-0.11005237766090797]

In [ ]:
# add this values to the results table

In [ ]:
results =  pd.read_csv('/content/drive/MyDrive/BSC/Ideas_on_counterfactuals/results_adult.csv')

In [ ]:
results['Lime Importance']= lime_importance

In [ ]:
# Riordina le colonne nel DataFrame
ordered_columns = ['Feature', 'Dissimilarity', 'DiCE Importance', 'SHAP Importance', 'Lime Importance', 'Pearson Correlation', 'Permutation Importance']

results = results[ordered_columns]

results


,Feature,Dissimilarity,DiCE Importance,SHAP Importance,Lime Importance,Pearson Correlation,Permutation Importance
0,age,0.525798,0.7180,-0.545029,-0.006652,0.233879,0.000537
1,hours_per_week,0.647884,0.8154,0.020406,-0.110052,0.225602,0.000403


In [ ]:
from google.colab import files

results.to_csv('results_adult.csv', index=False)
files.download('results_adult.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>